In [9]:
from astropy.io import fits

# Load the FITS file
fits_file = "mosaic_image.fits"  # Replace with your actual file
hdul = fits.open(fits_file)
header = hdul[0].header  # Access the primary header

# Extract pixel scale (CDELT values are in degrees)
pixel_scale_x = abs(header['CDELT1']) * 3600  # Convert degrees to arcsec
pixel_scale_y = abs(header['CDELT2']) * 3600  # Convert degrees to arcsec

# Image size in pixels
image_size_x = header['NAXIS1']
image_size_y = header['NAXIS2']

# Beam information (if available)
beam_major = header.get('BMAJ', 'N/A')  # In degrees
beam_minor = header.get('BMIN', 'N/A')
beam_pa = header.get('BPA', 'N/A')  # Position angle

if beam_major != 'N/A':
    beam_major *= 3600  # Convert to arcsec
    beam_minor *= 3600  # Convert to arcsec

# Print results
print(f"Pixel Resolution: {pixel_scale_x:.2f} × {pixel_scale_y:.2f} arcsec")
print(f"Image Size: {image_size_x} × {image_size_y} pixels")
print(f"Beam Size: {beam_major} × {beam_minor} arcsec, PA: {beam_pa} degrees")

hdul.close()


Pixel Resolution: 4.10 × 4.10 arcsec
Image Size: 6077 × 6074 pixels
Beam Size: 40.00000003092 × 22.0000000047012 arcsec, PA: 23.00000000004 degrees


In [10]:
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS

def process_fits(image_filename):
    """Reads a FITS file, extracts the 2D image data, and finds the peak flux density."""
    
    with fits.open(image_filename, mode='readonly') as hdu:
        header = hdu[0].header
        wcs = WCS(header, naxis=2)
        original_data = hdu[0].data.astype(np.float64)
    
    # Extract the correct 2D image slice
    if original_data.ndim == 4:
        base_data = original_data[0, 0, :, :]
    elif original_data.ndim == 3:
        base_data = original_data[0, :, :]
    elif original_data.ndim == 2:
        base_data = original_data.copy()
    else:
        raise ValueError(f"Unexpected FITS data dimensions: {original_data.shape}")
    
    # Check data shape
    print(f"Original data shape: {original_data.shape}")
    print(f"Processed image data shape: {base_data.shape}")
    
    # Handle NaN values
    base_data = np.nan_to_num(base_data)
    
    # Compute peak flux density
    peak_flux = np.nanmax(base_data)
    mini_flux = np.nanmin(base_data)
    print(f"Peak flux density: {peak_flux:.4f} Jy/beam")
    print(f"Minimum flux density: {mini_flux:.4f} Jy/beam")
    
    return peak_flux, mini_flux

# Example usage
peak_flux, mini_flux = process_fits(fits_file)

Set OBSGEO-B to    19.090653 from OBSGEO-[XYZ].
Set OBSGEO-H to      636.997 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


Original data shape: (1, 6074, 6077)
Processed image data shape: (6074, 6077)
Peak flux density: 14.3079 Jy/beam
Minimum flux density: -0.1329 Jy/beam


In [8]:
import numpy as np
from astropy.io import fits
from scipy.stats import norm
import matplotlib.pyplot as plt



def estimate_rms_gaussian_fit(image_data, clip_positive_sigma=5.0, bins=20):
    flat_data = image_data.flatten()
    flat_data = flat_data[np.isfinite(flat_data)]  # remove NaNs

    # Estimate robust median and std (initially)
    median = np.median(flat_data)
    mad = np.median(np.abs(flat_data - median))
    # rough_std = 1.4826 * mad  # for Gaussian distributions
    rough_std = mad  # for Gaussian distributions
    # rough_std = np.nanstd(flat_data)
    
    # Clip only **positive** outliers (symmetric clipping would hurt)
    clipped = flat_data[flat_data < median + clip_positive_sigma * rough_std]

    # # Histogram the clipped data
    # hist_vals, bin_edges = np.histogram(clipped, bins=bins, density=True)
    # bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

    # # Fit Gaussian to the histogram
    # popt, _ = norm.fit(clipped)
    # mu, std = popt, clipped.std()

    mu, std = np.mean(clipped), np.std(clipped)

    # # Optional: plot histogram with Gaussian
    # plt.hist(clipped, bins=bins, density=True, alpha=0.5, label='Clipped data')
    # x_vals = np.linspace(mu - 5*std, mu + 5*std, 500)
    # plt.plot(x_vals, norm.pdf(x_vals, mu, std), label='Gaussian fit')
    # plt.legend()
    # plt.title('Pixel Histogram and Gaussian Fit')
    # plt.xlabel('Pixel Value (Jy/beam)')
    # plt.show()

    return std  # This is your background RMS

# Usage:

PCs = ['AAAA', 'BBBB', 'CCCC', 'DDDD', 'EEEE', 'FFFF', 'GGGG']

for pc in PCs:
    imagefits = f'/media/asif/4TBHDD/BAND2-all/GSB/FIELDS/Allfields_mosaic/{pc}/fits/{pc}.SP2B.RES.FITS'
    image_data = fits.getdata(imagefits)
    rms_fit = estimate_rms_gaussian_fit(image_data)
    print(f"Estimated background RMS (Gaussian fit): {1e3 * rms_fit:.3f} mJy/beam")



Estimated background RMS (Gaussian fit): 3.346 mJy/beam
Estimated background RMS (Gaussian fit): 2.818 mJy/beam
Estimated background RMS (Gaussian fit): 2.421 mJy/beam
Estimated background RMS (Gaussian fit): 2.784 mJy/beam
Estimated background RMS (Gaussian fit): 2.741 mJy/beam
Estimated background RMS (Gaussian fit): 3.380 mJy/beam
Estimated background RMS (Gaussian fit): 2.773 mJy/beam
